# Introduction to STScI Data in AWS
### This notebook shows how to locate and obtain data from STScI's AWS public datasets. These files are free to download and access, which does not require AWS credentials.

In [1]:
import s3fs   #s3fs is a simple Python interface to the AWS S3 service.
import astroquery   #Astroquery is a Python interface to multiple astronomical archives and data services, including MAST and MAST AWS Public Data

### STScI's AWS public data are stored in AWS S3 buckets, which can be accessed with various client libraries depending on your preferred mode of searching for and obtaining files. See https://registry.opendata.aws/collab/stsci/ for more details.
### First, we use S3FS to list the directories and files associated with the TESS mission data.

In [2]:
fs = s3fs.S3FileSystem(anon=True)
print(s3fs.__version__)
fs.ls('s3://stpubdata/tess/public/',refresh=True)

0.5.2


['stpubdata/tess/public/',
 'stpubdata/tess/public/engineering',
 'stpubdata/tess/public/ffi',
 'stpubdata/tess/public/manifest.txt.gz',
 'stpubdata/tess/public/mast',
 'stpubdata/tess/public/models',
 'stpubdata/tess/public/pixel_list',
 'stpubdata/tess/public/tid']

### TESS mission data are organized under a directory structure type/sector/year/day/camera-chip

In [3]:
#list data files
fs.ls('s3://stpubdata/tess/public/ffi/s0010/2019/085/1-4/',refresh=True)

['stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-1-4-0140-s_cbv.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-lvcol-1-4-a-0140-s_col.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-lvcol-1-4-b-0140-s_col.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-lvcol-1-4-c-0140-s_col.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-lvcol-1-4-d-0140-s_col.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-smrow-1-4-a-0140-s_col.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-smrow-1-4-b-0140-s_col.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-smrow-1-4-c-0140-s_col.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-smrow-1-4-d-0140-s_col.fits',
 'stpubdata/tess/public/ffi/s0010/2019/085/1-4/tess2019085135100-s0010-tvcol-1-4-a-0140-s_col.fits'

In [4]:
#last modified
fs.modified('s3://stpubdata/tess/public/mast/tess-s0034-3-1-cube.fits',refresh=True)

datetime.datetime(2021, 3, 13, 5, 43, 49)

In [5]:
#display file size in MB
size=fs.du('s3://stpubdata/tess/public/ffi/s0031/2020/294/4-1/tess2020294194027-s0031-vrow-4-1-c-0198-a_fast-col.fits')/1024/1024
print(size)

1.2744140625


In [6]:
#Obtain a file locally
f=fs.get('s3://stpubdata/tess/public/ffi/s0031/2020/294/4-1/tess2020294194027-s0031-vrow-4-1-c-0198-a_fast-col.fits','tess_test.fits')

### Some auxiliary data is available, including cubes created by MAST for services such as TESScut 

In [7]:
#list TESSCut Cube FITS files in AWS
fs.ls('s3://stpubdata/tess/public/mast/',refresh=True)

['stpubdata/tess/public/mast/',
 'stpubdata/tess/public/mast/tess-s0001-1-1-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-1-2-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-1-3-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-1-4-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-2-1-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-2-2-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-2-3-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-2-4-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-3-1-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-3-2-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-3-3-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-3-4-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-4-1-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-4-2-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-4-3-cube.fits',
 'stpubdata/tess/public/mast/tess-s0001-4-4-cube.fits',
 'stpubdata/tess/public/mast/tess-s0002-1-1-cube.fits',
 'stpubdata/tess

## We can also use astroquery.MAST to search and filter data products, return the S3 paths, and download them.
## For astroquery >= 0.4.2, no AWS account is required.

In [8]:
from astroquery.mast import Observations

#Identify a few Sector 10 FFIs
obsTable = Observations.query_criteria(obs_id=f"tess-s0010-4-1")
products = Observations.get_product_list(obsTable)
filtered = Observations.filter_products(products[0:10], 
                                        productSubGroupDescription="FFIC",
                                        mrp_only=False)
f"Found {len(filtered)} products"


'Found 5 products'

In [9]:
#Return the AWS S3 locations (URIs)
Observations.enable_cloud_dataset(provider='AWS')
uris = Observations.get_cloud_uris(filtered)
print(uris[0])

INFO: Using the S3 STScI public dataset [astroquery.mast.cloud]
s3://stpubdata/tess/public/ffi/s0010/2019/085/4-1/tess2019085222934-s0010-4-1-0140-s_ffic.fits


In [10]:
#Download a few example products
manifest = Observations.download_products(products[0:2], cloud_only=True)

In [11]:
#Print local file location
print(manifest)

                                             Local Path                                             ...
--------------------------------------------------------------------------------------------------- ...
./mastDownload/TESS/tess2019085222934-s0010-4-1-0140-s/tess2019085222934-s0010-4-1-0140-s_ffic.fits ...
./mastDownload/TESS/tess2019085222934-s0010-4-1-0140-s/tess2019085222934-s0010-4-1-0140-s_ffir.fits ...
